In [2]:
import pandas as pd
import psycopg2
from datetime import datetime, time, timedelta
 

In [3]:
HOST = "rc1a-p8bp15mmxsfwpbt0.mdb.yandexcloud.net"
DB = "db1"
USER = "test_user"
PASSWORD = "j2M{CnnFq@"
PORT = "6432"

In [38]:
try:
    conn = psycopg2.connect(
        host=HOST,
        port=PORT,
        database=DB,
        user=USER,
        password=PASSWORD
    )
    print("Connection successful")
except Exception as e:
    print(f"Error: {e}")   

Connection successful


In [39]:
managers_query = "SELECT * FROM test.managers"
chat_messages_query = "SELECT * FROM test.chat_messages"
rops_query = "SELECT * FROM test.rops"

managers = pd.read_sql_query(managers_query, conn)
chat_messages = pd.read_sql_query(chat_messages_query, conn)
rops = pd.read_sql_query(rops_query, conn)
conn.close()

C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  managers = pd.read_sql_query(managers_query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chat_messages = pd.read_sql_query(chat_messages_query, conn)
C:\Users\user\AppData\Local\Temp\ipykernel_6604\1375247572.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rops = pd.read_sql_query(rops_query, conn)


In [58]:
chat_messages['created'] = pd.to_datetime(chat_messages['created_at'], unit='s')

In [59]:
def change_time(created):
    if created.hour < 9 or (created.hour == 9 and created.minute < 30):
        return created.replace(hour=9, minute=30, second=0, microsecond=0)
    else:
        return created
chat_messages['created'] = chat_messages['created'].apply(change_time)

In [60]:
chat_messages.head()

,group,message_id,type,entity_id,created_by,created_at,created,time_response
2,3,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2024-10-26 15:34:46,0 days 00:13:41
3,4,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46,0 days 00:02:00
8,9,"""01jb72hktgm0tvdpwk4a17dadz""",incoming_chat_message,36099337,0,1730035634,2024-10-27 13:27:14,0 days 00:01:27
9,10,"""01jb72nzkrzhcfhg9hfpbjt0vx""",outgoing_chat_message,36099337,11241942,1730035777,2024-10-27 13:29:37,0 days 00:02:23
15,16,"""01jb4v0dxgwsz4cqxg6wted3s2""",incoming_chat_message,36333761,0,1729960622,2024-10-26 16:37:02,0 days 00:03:34


In [61]:
chat_messages = chat_messages.sort_values(by=['entity_id', 'created'])
chat_messages['group'] = ((chat_messages['entity_id'] != chat_messages.shift()['entity_id']) | (chat_messages['type'] != chat_messages.shift()['type'])).cumsum()

In [62]:
chat_messages = chat_messages.groupby(by='group').first().reset_index()

In [63]:
chat_messages

,group,message_id,type,entity_id,created_by,created_at,created,time_response
0,1,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886,2024-10-26 15:34:46,0 days 00:13:41
1,2,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46,0 days 00:02:00
2,3,"""01jb72hktgm0tvdpwk4a17dadz""",incoming_chat_message,36099337,0,1730035634,2024-10-27 13:27:14,0 days 00:01:27
3,4,"""01jb72nzkrzhcfhg9hfpbjt0vx""",outgoing_chat_message,36099337,11241942,1730035777,2024-10-27 13:29:37,0 days 00:02:23
4,5,"""01jb4v0dxgwsz4cqxg6wted3s2""",incoming_chat_message,36333761,0,1729960622,2024-10-26 16:37:02,0 days 00:03:34
...,...,...,...,...,...,...,...,...
12664,12665,"""01jb7w2y40g1w0qck8ft1rywkt""",incoming_chat_message,37557147,0,1730062416,2024-10-27 20:53:36,0 days 00:00:49
12665,12666,"""01jb7w3c0by46wxq6mzsy536kz""",outgoing_chat_message,37557147,7996770,1730062430,2024-10-27 20:53:50,0 days 00:00:14
12666,12667,"""01jb7w4qqryt4tcd8xrv06c02s""",incoming_chat_message,37557147,0,1730062475,2024-10-27 20:54:35,0 days 00:00:45
12667,12668,"""01jb7w4sy7h12zmvst9s790vds""",outgoing_chat_message,37557147,7996770,1730062477,2024-10-27 20:54:37,0 days 00:00:02


In [64]:
chat_messages['time_response'] = chat_messages.groupby('entity_id')['created'].diff() 
chat_messages = chat_messages.dropna()

In [66]:
messages_response = chat_messages[chat_messages['type'] == 'outgoing_chat_message']
messages_response

,group,message_id,type,entity_id,created_by,created_at,created,time_response
1,2,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46,0 days 00:02:00
3,4,"""01jb72nzkrzhcfhg9hfpbjt0vx""",outgoing_chat_message,36099337,11241942,1730035777,2024-10-27 13:29:37,0 days 00:02:23
5,6,"""01jb4vbfpfwe4my2w5zhadeash""",outgoing_chat_message,36333761,10262485,1729960984,2024-10-26 16:43:04,0 days 00:06:02
7,8,"""01jb4ve204kyab9ez1hb5d2e1f""",outgoing_chat_message,36333761,10262485,1729961068,2024-10-26 16:44:28,0 days 00:01:01
9,10,"""01jb4vhn2hpdj09h1wre5ctzkn""",outgoing_chat_message,36333761,10262485,1729961186,2024-10-26 16:46:26,0 days 00:01:11
...,...,...,...,...,...,...,...,...
12659,12660,"""01jb7vtysr4g8ccm6bw9jxe1ar""",outgoing_chat_message,37557147,7996770,1730062154,2024-10-27 20:49:14,0 days 00:00:40
12661,12662,"""01jb7vyhhsyeezsr1pjs7k2thj""",outgoing_chat_message,37557147,7996770,1730062272,2024-10-27 20:51:12,0 days 00:01:06
12663,12664,"""01jb7w1f5fg44x88hdp0wbb3fx""",outgoing_chat_message,37557147,7996770,1730062367,2024-10-27 20:52:47,0 days 00:00:14
12665,12666,"""01jb7w3c0by46wxq6mzsy536kz""",outgoing_chat_message,37557147,7996770,1730062430,2024-10-27 20:53:50,0 days 00:00:14


In [69]:
messages_response['time_response']

1       0 days 00:02:00
3       0 days 00:02:23
5       0 days 00:06:02
7       0 days 00:01:01
9       0 days 00:01:11
              ...      
12659   0 days 00:00:40
12661   0 days 00:01:06
12663   0 days 00:00:14
12665   0 days 00:00:14
12667   0 days 00:00:02
Name: time_response, Length: 6182, dtype: timedelta64[ns]

In [ ]:
messages_response['time_response'] = messages_response['time_response'].dt.total_seconds()

In [73]:
messages_response

,group,message_id,type,entity_id,created_by,created_at,created,time_response
1,2,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006,2024-10-26 15:36:46,120.0
3,4,"""01jb72nzkrzhcfhg9hfpbjt0vx""",outgoing_chat_message,36099337,11241942,1730035777,2024-10-27 13:29:37,143.0
5,6,"""01jb4vbfpfwe4my2w5zhadeash""",outgoing_chat_message,36333761,10262485,1729960984,2024-10-26 16:43:04,362.0
7,8,"""01jb4ve204kyab9ez1hb5d2e1f""",outgoing_chat_message,36333761,10262485,1729961068,2024-10-26 16:44:28,61.0
9,10,"""01jb4vhn2hpdj09h1wre5ctzkn""",outgoing_chat_message,36333761,10262485,1729961186,2024-10-26 16:46:26,71.0
...,...,...,...,...,...,...,...,...
12659,12660,"""01jb7vtysr4g8ccm6bw9jxe1ar""",outgoing_chat_message,37557147,7996770,1730062154,2024-10-27 20:49:14,40.0
12661,12662,"""01jb7vyhhsyeezsr1pjs7k2thj""",outgoing_chat_message,37557147,7996770,1730062272,2024-10-27 20:51:12,66.0
12663,12664,"""01jb7w1f5fg44x88hdp0wbb3fx""",outgoing_chat_message,37557147,7996770,1730062367,2024-10-27 20:52:47,14.0
12665,12666,"""01jb7w3c0by46wxq6mzsy536kz""",outgoing_chat_message,37557147,7996770,1730062430,2024-10-27 20:53:50,14.0


In [83]:
mean_seconds_work = messages_response.groupby(by='created_by')['time_response'].mean().round()

In [86]:
result = pd.merge(managers, mean_seconds_work, how='inner', left_on = 'mop_id', right_on = 'created_by')


In [95]:
result['rop_id'] = result['rop_id'].astype(int)

In [96]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   mop_id         22 non-null     int64  
 1   name_mop       22 non-null     object 
 2   rop_id         22 non-null     int64  
 3   time_response  22 non-null     float64
dtypes: float64(1), int64(2), object(1)
memory usage: 836.0+ bytes


In [97]:
result = pd.merge(result, rops, how = 'inner')

In [102]:
result = result[['name_mop', 'time_response', 'rop_name']].sort_values(by='time_response')
result.columns = ['Менеджеры', 'Среднее время ответа', 'Фирма']
result

,Менеджеры,Среднее время ответа,Фирма
15,Алина и Юля,207.0,Эля РОП
13,Влада и Настя,235.0,Эля РОП
7,Даша и Даша,236.0,Полина РОП
16,Ками и Мила,237.0,Эля РОП
21,Ангелина Милованова,238.0,Эля РОП
20,Софья Боднар,280.0,Эля РОП
3,Настя и Даша,304.0,Катя РОП
18,Даша и Карина,347.0,Эля РОП
6,Ксюша и Джамиля,364.0,Катя РОП
8,Лиза и Ева,372.0,Полина РОП
